#Hard coded values
Column names
'createdatl,'updatedatl','marketproduct_id','Eddress ID','product_barcode','marketproduct_name_de','segment_name',
'brand_name','category_name_en','subcategory_name_en','supplier_names','list_price,'g10_price','transfer_groups','isvisible'

Segment filter
"Asset", "Delist", "None", "Test", "Only Promo"

category managers
"Marius","Fabian","Gina","Hamid","Amadeus","Rozan","Francisco","Eike","Evrim","Leonard","Andrea"

Cities
"Berlin","Berlin Mini", "Dresden","Leipzig","Hamburg","Bremen","Hannover","Düsseldorf","Dortmund","Köln","Bonn","Essen","München", "Nürnberg/Fürth","Frankfurt","Offenbach","Stuttgart","Heidelberg","Mannheim","Karlsruhe","Darmstadt","Augsburg"

segments	segment_name
1	Test
10	Seasonal
11	Supply Shortage
12	Stand By
2	Launch
3	Regular
4	Regional
5	Only Promo
6	Delist On Depletion
7	Delist
8	Asset


final output columns format


#Using Instructions
1 - save panel report on desktop and rename until date. (ex - G10andGetirBuyuk_AllProducts_without_TRDE_20230605.xlsx)
2 - save eddress export all on desktop and rename as Inventory.xls
3 - run the code - you will have a sheet on your desktop "New_assortment"
4 - copy the data from "New_assortment" and paste it in Assortment list sheet - "Weekly updated Assortment List"





In [35]:
#import packages
import pandas as pd
import numpy as np
from datetime import datetime as dt
from datetime import date
today = dt.now()
mask = '%Y%m%d'
dte = dt.now().strftime(mask)

#Read the downloaded report
Path_A = "/Users/swaroopbasavarajmusti/Desktop/G10andGetirBuyuk_AllProducts_without_TRDE.xlsx"

Assortment = pd.read_excel(Path_A)

#change the marketplace product id column name (not required but to avoid confusion)

Assortment.rename(columns={'marketproduct_id':'Getir Product Id'}, inplace=True)

#select only required columns

Assortment = Assortment[['createdatl','updatedatl','Getir Product Id','product_barcode','marketproduct_name_de','segment_name',
'brand_name','category_name_en','subcategory_name_en','supplier_names','net_net_buying_price_with_vat','g10_price','transfer_groups','isvisible',\
'isbundle','exp_days_lifetime','exp_days_allowed','exp_days_warning','exp_days_dead','inbox_quantitiy',\
'palet_inbox_quantity','storagetype','transfer_coli_count','is_picked_to_zero','market_status']]

#filter unwanted segments from the dataframe
Segment_filters = ["Asset", "None", "Test", "Only Promo"]
Assortment = Assortment[~Assortment.segment_name.isin(Segment_filters)].reset_index(drop = True)

#sort the values based on the date created
Assortment = Assortment.sort_values(by="createdatl",ascending = False).reset_index(drop = True)

#------------------------------

#creating dictionary to match the product ids

Path_B = "/Users/swaroopbasavarajmusti/Desktop/Inventory.xls"
Dictionary = pd.read_excel(Path_B)

Dictionary = Dictionary[Dictionary['Getir Product Id'].notna()].reset_index(drop = True)
Dictionary = Dictionary[['Getir Product Id','Product Id','Product SKU','Enabled / Disabled']]


#------------------------------

#left joining Panel assortment with eddress ID dictionary

Assortment_Inventory = pd.merge(Assortment, 
                      Dictionary, 
                      on ='Getir Product Id', 
                      how ='left')

#------------------------------

# read purchase order report 

Path_C = "/Users/swaroopbasavarajmusti/Desktop/G10_and_GB_purchaseorderInfo_20230731.xlsx"
PO = pd.read_excel(Path_C)

#rename primary keys and reduce to only required columns
PO.rename(columns={'Item':'Getir Product Id'}, inplace=True)
PO.rename(columns={'Job Status Name':'Job_status'}, inplace=True)

#filter unwanted columns from the dataframe and create only required dataframe
PO = PO[['Getir Product Id','Purchaseorder Createdat L','Job_status']]

#filter unwanted job status from the dataframe and create only required dataframe
Job_Status_Name_filters = ["CANCELLED"]
PO = PO[~PO.Job_status.isin(Job_Status_Name_filters)].reset_index(drop = True)

#look for unique values of products ordered and create ordered status
Ordered_products = pd.DataFrame(data=PO['Getir Product Id'].unique(), columns=["Getir Product Id"])
Ordered_products['Order_Status'] = 'TRUE'

#------------------------------

#join the two table with primary
Assortment_Joined = pd.merge(Assortment_Inventory, 
                      Ordered_products, 
                      on ='Getir Product Id', 
                      how ='left')

#------------------------------

#filtering only required columns
Assortment_Joined = Assortment_Joined[['createdatl','updatedatl','Getir Product Id','Product Id','Product SKU','product_barcode','marketproduct_name_de','segment_name',
'brand_name','category_name_en','subcategory_name_en','supplier_names','net_net_buying_price_with_vat','g10_price','isvisible','transfer_groups','Enabled / Disabled','Order_Status',\
'isbundle','exp_days_lifetime','exp_days_allowed','exp_days_warning','exp_days_dead','inbox_quantitiy',\
'palet_inbox_quantity','storagetype','transfer_coli_count','is_picked_to_zero','market_status']]

#convert datetime format to just date
Assortment_Joined['createdatl'] = Assortment_Joined['createdatl'].dt.date
Assortment_Joined['updatedatl'] = Assortment_Joined['updatedatl'].dt.date

#------------------------------

#create categories and categories manager dictionary
Category_dictionary = {\
"Rozan":['Coffee & Tea'],
"Marius":['Baby','Beer','Home Care','Personal Care','Pets','Spirits & Co.','Wine & Co.'],
"Leonard":['Fruit & Veg'],
"Hamid":['Cheese & Co.','Milk & Eggs','Yoghurt & Co.','Cold Cuts','Meat & Fish'],
"Francisco":['Fresh & Ready'],
"Fabian":['Bakery'],
"Evrim":['Frozen Food','Ice Cream'],
"Eike":['Drinks','Breakfast','Pantry','Pasta & Rice','Canned','Crisps & Snacks'],
"Amadeus":['Christmas','Easter','Sweet Snacks'],
"Other":['0','DUMMY TEST KATEGORI GE',"Mother's Day",'Top Deals','Earthquake Aid','Dairy','Cheese & Deli','Vegan & Veggie','Confectionery','Vegan']}

#------------------------------

#match the categories and get the name of category manager
for ind, row in Assortment_Joined.iterrows():
    if Assortment_Joined.loc[ind, 'category_name_en'] == "":
        Assortment_Joined.loc[ind, 'Buyer'] = "Not Assigned"
    else:
        Category = Assortment_Joined.loc[ind, 'category_name_en']
        Assortment_Joined.loc[ind, 'Buyer'] = [k for k, v in Category_dictionary.items() if Category in v]


# #get only the name without brackets
#Assortment_Joined['Buyer'] = ''
Assortment_Joined['Buyer'] = Assortment_Joined['Buyer'].astype(str)
Assortment_Joined['Buyer'] = Assortment_Joined['Buyer'].str[2:-2]


#------------------------------



#change the eddress enabled/disabled status to true and false
for ind,row in Assortment_Joined.iterrows():
    if Assortment_Joined.loc[ind, 'Enabled / Disabled'] == 'Enabled':
        Assortment_Joined.loc[ind, 'Live Gorillas app'] = "TRUE"
    else:
        Assortment_Joined.loc[ind, 'Live Gorillas app'] = "FALSE"
        



#------------------------------

#Generate the y/n combination for transfer group(mfc) checks
Assortment_Joined['transfer_groups'] = Assortment_Joined['transfer_groups'].astype(str)

#create a list of cities 
cities = ["Berlin","Berlin mini", "Dresden","Leipzig","Hamburg","Bremen",\
          "Hannover","Düsseldorf","Dortmund","Köln","Bonn","Essen","München",\
          "NürnbergFürth","Frankfurt","Offenbach","Stuttgart","Heidelberg",\
          "Mannheim","Karlsruhe","Darmstadt","Augsburg"]

Assortment_Joined['Cities'] = ''
#iterate over transfer groups to check for cities
for c in cities:
    for ind, row in Assortment_Joined.iterrows():
        x = "merged "+c
        if c in Assortment_Joined.loc[ind, 'transfer_groups']:
            Assortment_Joined.loc[ind, c] = "Y"
            a = Assortment_Joined.loc[ind, 'Cities']
            b = c
            new_Cit = a+", "+b
            Assortment_Joined.loc[ind, 'Cities'] = new_Cit
        else:
            Assortment_Joined.loc[ind, c] = "N"
            
Assortment_Joined['Cities'] = Assortment_Joined['Cities'].str[2:]
#------------------------------

#renaming and del column
Assortment_Joined.rename(columns={'createdatl':'Product Card Creation Date'}, inplace=True)
Assortment_Joined.rename(columns={'updatedatl':'Update date'}, inplace=True)
Assortment_Joined.rename(columns={'net_net_buying_price_with_vat':'Buying Price (w VAT)'}, inplace=True)
Assortment_Joined.rename(columns={'g10_price':'Selling Price (w VAT)'}, inplace=True)
Assortment_Joined.rename(columns={'isvisible':'Live Getir app'}, inplace=True)
Assortment_Joined.rename(columns={'Product Id':'Eddress Product Id'}, inplace=True)
Assortment_Joined.rename(columns={'product_barcode':'GTIN_Panel'}, inplace=True)
Assortment_Joined.rename(columns={'Product SKU':'GTIN_Eddress'}, inplace=True)
Assortment_Joined.rename(columns={'segment_name':'Segment_Panel'}, inplace=True)

#------------------------------
#additional formatting
Assortment_Joined['Update date'] = ''
del Assortment_Joined['transfer_groups']
Assortment_Joined['Order_Status'] = Assortment_Joined['Order_Status'].fillna('FALSE')
del Assortment_Joined['Enabled / Disabled']
        
#------------------------------

#sorting and reset index
Assortment_Joined.sort_values(by='Product Card Creation Date', ascending = False ,inplace = True)
Assortment_Joined = Assortment_Joined.reset_index(drop=True)

#-------------------------------

#adding mdm data to dataset to get the product status on mdm

Path_D = "/Users/swaroopbasavarajmusti/Desktop/mdm_data.xlsm"
mdm_data_all = pd.read_excel(Path_D,'Entities')
mdm_data_all = mdm_data_all.rename(columns=mdm_data_all.iloc[0])
mdm_data_all = mdm_data_all.iloc[1:].reset_index(drop = True)
mdm_data = mdm_data_all.copy()
mdm_data = mdm_data[['Eddress Product ID','Web Pages.File Name','Product Status']]
mdm_data = mdm_data[mdm_data['Eddress Product ID'].notna()].reset_index(drop =True)
mdm_data.rename(columns={'Eddress Product ID':'Eddress Product Id'}, inplace=True)
mdm_data.rename(columns={'Product Status':'Segment_MDM'}, inplace=True)
mdm_data = mdm_data.drop_duplicates(subset=['Eddress Product Id'], keep='first')
check1 = mdm_data.copy()

Merged_assortment = pd.merge(Assortment_Joined, 
                      mdm_data, 
                      on ='Eddress Product Id', 
                      how ='left')

#-------------------------------
#checking if the product is created on eddress by creating a column

for ind,row in Merged_assortment.iterrows():
    if pd.isnull(Merged_assortment.loc[ind, 'Eddress Product Id']) == True:
        Merged_assortment.loc[ind, 'Created on Eddress'] = "FALSE"
    else:
        Merged_assortment.loc[ind, 'Created on Eddress'] = "TRUE"

#-------------------------------

#create segment dictionary and segment attributes to sync statuses
Segment_Dictionary = {\

"PUBLISHED":['Regular','Regional'],
"UNPUBLISHED - New Ingoing":['Launch'],
"PUBLISHED - Deplete":['Seasonal','Only Promo','Delist On Depletion','Consumable'],
"UNAVAILABLE - OOS Supplier":['Stand By','Supply Shortage'],
"UNPUBLISHED - Delisted":['Delist','None','Asset','Test']

}

#create new segment statuts based on dictionary
for ind, row in Merged_assortment.iterrows():
    if pd.isnull(Merged_assortment.loc[ind, 'Segment_Panel']) == True:
        Merged_assortment.loc[ind, 'Segment_New'] = ""
    else:
        segment = str(Merged_assortment.loc[ind, 'Segment_Panel'])
        Merged_assortment.loc[ind, 'Segment_New'] = [k for k, v in Segment_Dictionary.items() if segment in v]

#get only the Segments without brackets
Merged_assortment['Segment_New'] = Merged_assortment['Segment_New'].astype(str)
Merged_assortment['Segment_New'] = Merged_assortment['Segment_New'].str[2:-2]

#create a filter column to filter products that have to be updated
for ind, row in Merged_assortment.iterrows():
    if pd.isnull(Merged_assortment.loc[ind, 'Segment_MDM']) == True:
        Merged_assortment.loc[ind, 'Segment_update'] = ""
    elif Merged_assortment.loc[ind, 'Segment_New'] == Merged_assortment.loc[ind, 'Segment_MDM']:
        Merged_assortment.loc[ind, 'Segment_update'] = False
    else:
        Merged_assortment.loc[ind, 'Segment_update'] = True

#-------------------------------
#vhange price volumns to euro currency format

Merged_assortment["Buying Price (w VAT)"] = Merged_assortment["Buying Price (w VAT)"].map("€ {:_.2f}".format).str.translate(str.maketrans("_.", ".,"))
Merged_assortment["Selling Price (w VAT)"] = Merged_assortment["Selling Price (w VAT)"].map("€ {:_.2f}".format).str.translate(str.maketrans("_.", ".,"))


#-------------------------------
#bring relistings and delistings dataframe and add it to main df
#path and load from desktop
Path_f = '/Users/swaroopbasavarajmusti/Desktop/productCardChange.xlsx'
metadata = pd.read_excel(Path_f)
today = date.today()

#filter for delist on depletion product with ids
Dod = metadata.copy()
Dod = Dod.loc[Dod['updated_field'] == 'segments']
Dod = Dod.sort_values(by="updated_at",ascending = False)
Dod = Dod.drop_duplicates(subset=['product_id'], keep='first')
Dod = Dod.loc[Dod['new_value'] == 6]
Dod['updated_at'] = Dod['updated_at'].dt.date
Dod['Date_filter'] = today - Dod['updated_at']
Dod['Date_filter'] = pd.to_timedelta(Dod.Date_filter, errors='coerce').dt.days
#Dod = Dod.loc[Dod['Date_filter'] < 14].reset_index(drop = True)
#create column as Action and fill in "Delist on depletion"
Dod['Action'] = 'Delist On Depletion'
Dod = Dod[['product_id','updated_at','Action']]

#filter for Relist product with ids
Rel = metadata.copy()
Rel = Rel.loc[Rel['updated_field'] == 'segments']
Rel = Rel.sort_values(by="updated_at",ascending = False)
Rel = Rel.drop_duplicates(subset=['product_id'], keep='first')
Rel = Rel.loc[Rel['old_value'] == 7]
nv_filter = [6]
Rel = Rel[~Rel.new_value.isin(nv_filter)].reset_index(drop = True)
Rel['updated_at'] = Rel['updated_at'].dt.date
Rel['Date_filter'] = today - Rel['updated_at']
Rel['Date_filter'] = pd.to_timedelta(Rel.Date_filter, errors='coerce').dt.days
#Rel = Rel.loc[Rel['Date_filter'] < 14].reset_index(drop = True)
#create column as Action and fill in "Relisting"
Rel['Action'] = 'Relisting'
Rel = Rel[['product_id','updated_at','Action']]

#2nd filter for Relist product with ids
Rel2 = metadata.copy()
Rel2 = Rel2.loc[Rel2['updated_field'] == 'segments']
Rel2 = Rel2.sort_values(by="updated_at",ascending = False)
Rel2 = Rel2.drop_duplicates(subset=['product_id'], keep='first')
Rel2 = Rel2.loc[Rel2['old_value'] == 6]
#Rel2 = Rel2.loc[Rel2['new_value'] == 2]
nv_filter = [2]
Rel2 = Rel2[Rel2.new_value.isin(nv_filter)].reset_index(drop = True)
Rel2['updated_at'] = Rel2['updated_at'].dt.date
Rel2['Date_filter'] = today - Rel2['updated_at']
Rel2['Date_filter'] = pd.to_timedelta(Rel2.Date_filter, errors='coerce').dt.days
#Rel = Rel.loc[Rel['Date_filter'] < 14].reset_index(drop = True)
#create column as Action and fill in "Relisting"
Rel2['Action'] = 'Relisting'
Rel2 = Rel2[['product_id','updated_at','Action']]

#filter for Delistings product with ids
Del = metadata.copy()
Del = Del.loc[Del['updated_field'] == 'segments']
Del = Del.sort_values(by="updated_at",ascending = False)
Del = Del.drop_duplicates(subset=['product_id'], keep='first')
#Del = Del.loc[Del['old_value'] == 6]
Del = Del.loc[Del['new_value'] == 7]
#nv_filter = [2]
#Del = Del[Del.new_value.isin(nv_filter)].reset_index(drop = True)
Del['updated_at'] = Del['updated_at'].dt.date
Del['Date_filter'] = today - Del['updated_at']
Del['Date_filter'] = pd.to_timedelta(Del.Date_filter, errors='coerce').dt.days
#Rel = Rel.loc[Rel['Date_filter'] < 14].reset_index(drop = True)
#create column as Action and fill in "Relisting"
Del['Action'] = 'Delisted'
Del = Del[['product_id','updated_at','Action']]

#Concat delist on depletions df and Relisting in one table and format
Updates = pd.concat([Dod,Rel, Rel2,Del],ignore_index=True).reset_index(drop=True)
Updates = Updates.sort_values(by="updated_at",ascending = False)
Updates = Updates.drop_duplicates(subset=['product_id'], keep='first')
Updates.rename(columns={'product_id':'Getir Product Id'}, inplace=True)

#merge with Main df to add the Action and update date to main df
weekly_assortment = pd.merge(Merged_assortment, 
                      Updates, 
                      on ='Getir Product Id', 
                      how ='left')

weekly_assortment['Update date'] = weekly_assortment['updated_at']


#-------------------------------

#organising the columns based on requirements
weekly_assortment = weekly_assortment[[\
#Update dates
'Action',\
'Product Card Creation Date',\
'Update date',\

#ids
'Getir Product Id',\
'Eddress Product Id',\

#segments
'Created on Eddress',\
'Segment_MDM',\
'Segment_Panel',\

#product statusus on multiple tools
'Live Getir app',\
'Live Gorillas app',\

#gtins details
'GTIN_Eddress',\
'GTIN_Panel',\
'marketproduct_name_de',\
'brand_name',\
'category_name_en',\
'subcategory_name_en',\
'supplier_names',\
'Buying Price (w VAT)',\
'Selling Price (w VAT)',\
'Buyer',\


#MFCs
'Berlin','Berlin mini','Dresden','Leipzig','Hamburg','Bremen','Hannover','Düsseldorf','Dortmund',\
'Köln','Bonn','Essen','München','NürnbergFürth','Frankfurt','Offenbach','Stuttgart','Heidelberg',\
'Mannheim','Karlsruhe','Darmstadt','Augsburg',\

#Cityjoin
'Cities',\
#filters
'Order_Status',\
    
#logistic details
'isbundle','exp_days_lifetime','exp_days_allowed','exp_days_warning','exp_days_dead','inbox_quantitiy',\
'palet_inbox_quantity','storagetype','transfer_coli_count','is_picked_to_zero','market_status'
]]



#-------------------------------

#Create a ouptut to excel
#weekly_assortment.to_excel("/Users/swaroopbasavarajmusti/Desktop/weekly_assortment.xlsx")



FileNotFoundError: [Errno 2] No such file or directory: '/Users/swaroopbasavarajmusti/Desktop/G10_and_GB_purchaseorderInfo_20230731.xlsx'

In [33]:
Path_G = '/Users/swaroopbasavarajmusti/Desktop/current_stock_info_20230731.xlsx'
stock_df = pd.read_excel(Path_G)

df = stock_df.copy()

df.rename(columns={'product_id':'Getir Product Id'}, inplace=True)

Closed_Warehouses = ['Alex','Aplerbeck','Barmbek_Gorillas','Berlin HUB','Bickendorf','Billstedt','Bispingen CW',\
           'Bispingen CW_NoNag','Boeblinger_Gorillas','Cologne - CHOCO','DE_Training_WH','Dortmund HUB - Choco',\
           'Dummy Berlin CW','Friedenau','Gbrunnen Fleet','Getir DE Defleet','Getir DE Office','Getir Germany Test',\
           'Haidhausen','Halensee','Harras','Jungfernheide','JungleTEST','Langwasser','Lichtenberg','Luft_Gorillas',\
           'MagnaPark CW_NoNag','Meringplatz','Munich HUB - Choco','Nordbahnhof','Nürnberg HUB - Choco','Ottensen',\
           'Pasing','Staaken','Stellingen','Tempelhof','Trudering','UNeukolln','Unterbilk','Unterrath','Velodrom',\
           'Wandsbek','Weissensee','Winsviertel','Wittenbergplatz','ZOBMesse','Zabo']
for C in Closed_Warehouses:
    df = df[df.warehouse_name != C]

warehouse_list = df.warehouse_name.unique().tolist()

df_1 = df.pivot_table(index=['Getir Product Id'],columns='warehouse_name',values = 'available',aggfunc='sum').\
                reset_index()

#df_1.drop(columns = Closed_Warehouses)

df_1['Total'] = int(0)
for w in warehouse_list:
    df_1[w]= df_1.fillna(0)[w].astype(int)
    df_1['Total'] = df_1['Total'] + df_1[w]

Merged_assortment_stock = pd.merge(Assortment, 
                      df_1, 
                      on ='Getir Product Id', 
                      how ='left')

Merged_assortment_stock['Stock_Value'] = Merged_assortment_stock['Total']*\
Merged_assortment_stock['net_net_buying_price_with_vat']
Merged_assortment_stock['Stock_Value'] = Merged_assortment_stock['Stock_Value'].fillna(0).astype(int)
Merged_assortment_stock['Profit/Product'] = Merged_assortment_stock['g10_price']-Merged_assortment_stock['net_net_buying_price_with_vat']
Merged_assortment_stock['Profit/Product'] = round(Merged_assortment_stock['Profit/Product'],2)
#cat_wise = Merged_assortment_stock.copy()

#cat_wise = cat_wise.groupby(['category_name_en','subcategory_name_en']).sum('Stock_Value').reset_index()
#cat_wise = cat_wise.sort_values('Stock_Value', ascending=False).reset_index(drop = True)

Path_H = '/Users/swaroopbasavarajmusti/Desktop/marketProductSales.xlsx'
Sales_df = pd.read_excel(Path_H)

Sales_df.rename(columns={'product_id':'Getir Product Id'}, inplace=True)
dict_of_Marketplaces = {k: v for k, v in Sales_df.groupby('integrations_breakdown')}
Marketplaces = Sales_df['integrations_breakdown'].unique()
Unique_ID_df = pd.DataFrame(Sales_df['Getir Product Id'].unique())
Unique_ID_df.rename(columns={0:'Getir Product Id'}, inplace=True)

All_means = pd.DataFrame()
for M in Marketplaces:
    df = dict_of_Marketplaces[M].copy()
    df = df.pivot_table(index=['Getir Product Id'],columns='week',values = 'items_sold_count',aggfunc='sum').\
                reset_index()
    temp_name = M +"_mean"
    df[temp_name] = df.mean(numeric_only=True, axis=1)
    df[temp_name] = df[temp_name].astype(int)
    df = df[['Getir Product Id', temp_name ]]
    Unique_ID_df = Unique_ID_df.merge(df, on ='Getir Product Id', how ='left')
    
Master = pd.merge(Merged_assortment_stock, 
                  Unique_ID_df, 
                  on ='Getir Product Id', 
                  how ='left')

import warnings
warnings.simplefilter("ignore")
from oauth2client.service_account import ServiceAccountCredentials
import gspread

scope = ['https://www.googleapis.com/auth/spreadsheets','https://www.googleapis.com/auth/drive.file', \
        'https://www.googleapis.com/auth/drive','https://www.googleapis.com/auth/drive.readonly']

creds = ServiceAccountCredentials.from_json_keyfile_name('panel-321812-ac5afc8690ea.json',scope)
client = gspread.authorize(creds)

Expiry_data  = client.open('Gsheet trial').worksheet('Stock')
Expiry_df = pd.DataFrame(Expiry_data.get_all_records())

Expiry_df['Today'] = dt.now().strftime('%Y%m%d')
Expiry_df['Today'] = pd.to_datetime(Expiry_df['Today'], errors='coerce') 

Expiry_df['Expire date'] = pd.to_datetime(Expiry_df['Expire date'], errors='coerce')

Expiry_df['Remaining_Days'] = Expiry_df['Expire date'] - Expiry_df['Today']
Expiry_df['Remaining_Days'] = pd.to_timedelta(Expiry_df.Remaining_Days, errors='coerce').dt.days

Expiry_df = Expiry_df[['Item code','Expire date','Today','Remaining_Days']]\
.sort_values(by = 'Remaining_Days', ascending=[True]).reset_index(drop = True)
Expiry_df.rename(columns={'Item code':'Getir Product Id'}, inplace=True)

Master_V2 = pd.merge(Master, 
                  Expiry_df, 
                  on ='Getir Product Id', 
                  how ='left')

Master_V2 = Master_V2.drop_duplicates(subset=['Getir Product Id', 'Remaining_Days'])

Analysis = Master_V2.copy()

with pd.ExcelWriter("/Users/swaroopbasavarajmusti/Desktop/Promotional_Products_{}.xlsx".format(dte)) as writer:
    for M in Marketplaces:
        Analysis['Cover_days_'+M] = (Analysis['Total']/Analysis[M+'_mean'])*7
        Analysis['Cover_days_'+M] = Analysis['Cover_days_'+M].fillna(0).astype(int)
        Analysis['STR_'+M] = round((Analysis[M+'_mean']/Analysis['Total'])*100, 2)
        Analysis['Risk_Rank_'+M] = round((Analysis['Cover_days_'+M] - Analysis['Remaining_Days']), 0).rank(ascending=False)
        Analysis['Risk_Rank_'+M] = Analysis['Risk_Rank_'+M].fillna(0).astype(int)
        
        temp_df = Analysis.copy()
        temp_df =  temp_df[(temp_df['Cover_days_'+M]>60)]
        temp_df = temp_df[['createdatl','Getir Product Id','product_barcode','marketproduct_name_de',\
                          'category_name_en','subcategory_name_en','segment_name','Total','Stock_Value','Profit/Product',\
                          M+'_mean','STR_'+M,'Cover_days_'+M,'Expire date','Remaining_Days','Risk_Rank_'+M]].sort_values(by="Cover_days_"+M,ascending = False).reset_index(drop = True)
        temp_df = temp_df.sort_values(by='Risk_Rank_'+M,ascending = True).reset_index(drop = True)
        temp_df.to_excel(writer, sheet_name = M)
        col = "Risk_Rank_"+M
        temp_df_2 = temp_df[(temp_df[col] > 0)& (temp_df[col] < 31)].reset_index(drop = True)
        temp_df_2.to_excel(writer, sheet_name = "top_30_"+M)

Gorillas Only Products: Private Label Products: Indy started a list here - let’s use some kind of dictionary like that: https://docs.google.com/spreadsheets/d/1HbBSPP495FKP2syAOkbMEPOGrlFozz0LzI2T0H1mV44/edit#gid=979825498
Getir only products: Define List (Chris)
Columns to add (middterm):
Local/Regional/National products: define rule depending on transfer groups